In [1]:
%pylab inline
from parcels import FieldSet, ParticleSet, JITParticle, Variable, AdvectionRK4
import numpy as np
import xarray as xr
from datetime import timedelta as delta
import matplotlib.pyplot as plt
from collections import defaultdict

Populating the interactive namespace from numpy and matplotlib


In [2]:
vels ={'total': ['utotal', 'vtotal'], 
       'curr': ['uo', 'vo'], 
       'stokes': ['vsdx', 'vsdy'], 
       'tide': ['utide', 'vtide']}

for veltype in vels:
    print(veltype)
    
    variables = {'U': vels[veltype][0], 'V': vels[veltype][1]}
    dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time', 'depth': 'depth'}
    fieldset = FieldSet.from_netcdf('/Users/erik/Desktop/SargassumData/TrAtlUV*.nc', variables, dimensions)

    data = np.load('spottrajs_200508.npz', allow_pickle=True)
    launch_lon = data['launch_lon'].item()
    launch_lat = data['launch_lat'].item()
    launch_times = data['launch_times'].item()
    sets = data['sets']
    for s in sets:
        for i in s[1:]:
            for var in [launch_lon, launch_lat, launch_times]:
                del var[i]
    launch_ids = list(launch_lon.keys())

    def AgeParticle(particle, fieldset, time):
        if particle.state == ErrorCode.Evaluate:
            particle.age += math.fabs(particle.dt)
        if particle.age > fieldset.maxage:
            particle.delete()

    class DrifterParticle(JITParticle):
        age = Variable('age')
        launch_id = Variable('launch_id', to_write='once')

    pset = ParticleSet(fieldset, pclass=DrifterParticle, lon=list(launch_lon.values()),
                       lat=list(launch_lat.values()), time=list(launch_times.values()), launch_id=launch_ids)

    fieldset.maxage = delta(days=5).total_seconds()
    runtime = (max(list(launch_times.values()))-min(list(launch_times.values()))).total_seconds() + fieldset.maxage

    pfname = '/Users/erik/Desktop/SargassumData/sargassum_parcels_%s.nc' %veltype
    pset.execute(pset.Kernel(AdvectionRK4)+AgeParticle, dt=delta(minutes=10), runtime=runtime,
                 output_file=pset.ParticleFile(pfname, outputdt=delta(hours=1)))


    pfile = xr.open_dataset(pfname, decode_cf=True)

    sets = [list([int(s)]) for s in launch_ids]

    lons = defaultdict(list)
    lats = defaultdict(list)
    dates = defaultdict(list)
    ids = pfile.variables['launch_id']
    for i in range(ids.shape[0]):
        pid = int(ids[i].values)
        lons[pid] = pfile.variables['lon'][i, :].values
        lats[pid] = pfile.variables['lat'][i, :].values
        dates[pid] = [datetime.datetime.utcfromtimestamp(d.astype('O')/1e9) for d in pfile.variables['time'][i, :].values if np.isfinite(d)]

    file='parcelstrajs_%s.npz' % veltype
    np.savez(file, lats=lats, lons=lons, dates=dates, sets=sets, 
             launch_lat=launch_lat, launch_lon=launch_lon, launch_times=launch_times)

total


INFO: Compiled DrifterParticleAdvectionRK4AgeParticle ==> /var/folders/r2/8593q8z93kd7t4j9kbb_f7p00000gr/T/parcels-504/688bb5b894427a89a61cd2dd86eaf457_0.so
INFO: Temporary output files are stored in /Users/erik/Desktop/SargassumData/out-PAVSVOQM.
INFO: You can use "parcels_convert_npydir_to_netcdf /Users/erik/Desktop/SargassumData/out-PAVSVOQM" to convert these to a NetCDF file during the run.
 21% (432000.0 of 2049540.0) |##         | Elapsed Time: 0:00:04 ETA:   0:01:07WARNING: Deleting particle because of bug in #749 and #737
 34% (698400.0 of 2049540.0) |###        | Elapsed Time: 0:00:17 ETA:   0:01:11WARNING: time argument in pfile.write() is 767760.0, but particles have time [767880.]
 61% (1263600.0 of 2049540.0) |######    | Elapsed Time: 0:00:52 ETA:   0:00:56WARNING: time argument in pfile.write() is 1332960.0, but particles have time [1333080.]
100% (2049540.0 of 2049540.0) |##########| Elapsed Time: 0:02:00 Time:  0:02:00


curr


INFO: Compiled DrifterParticleAdvectionRK4AgeParticle ==> /var/folders/r2/8593q8z93kd7t4j9kbb_f7p00000gr/T/parcels-504/c8b23ccdcd33a3ca62a4d82a6f45dcf1_0.so
INFO: Temporary output files are stored in /Users/erik/Desktop/SargassumData/out-IXQWIYRQ.
INFO: You can use "parcels_convert_npydir_to_netcdf /Users/erik/Desktop/SargassumData/out-IXQWIYRQ" to convert these to a NetCDF file during the run.
100% (2049540.0 of 2049540.0) |##########| Elapsed Time: 0:02:01 Time:  0:02:01


stokes


INFO: Compiled DrifterParticleAdvectionRK4AgeParticle ==> /var/folders/r2/8593q8z93kd7t4j9kbb_f7p00000gr/T/parcels-504/3df7d742d754d1c8dbfe7045c50faefb_0.so
INFO: Temporary output files are stored in /Users/erik/Desktop/SargassumData/out-DHOUFPHN.
INFO: You can use "parcels_convert_npydir_to_netcdf /Users/erik/Desktop/SargassumData/out-DHOUFPHN" to convert these to a NetCDF file during the run.
100% (2049540.0 of 2049540.0) |##########| Elapsed Time: 0:01:58 Time:  0:01:58


tide


INFO: Compiled DrifterParticleAdvectionRK4AgeParticle ==> /var/folders/r2/8593q8z93kd7t4j9kbb_f7p00000gr/T/parcels-504/5fd15754dfc4db1d8311749b3f5d0cd2_0.so
INFO: Temporary output files are stored in /Users/erik/Desktop/SargassumData/out-LGMZJNMP.
INFO: You can use "parcels_convert_npydir_to_netcdf /Users/erik/Desktop/SargassumData/out-LGMZJNMP" to convert these to a NetCDF file during the run.
100% (2049540.0 of 2049540.0) |##########| Elapsed Time: 0:01:58 Time:  0:01:58
